In [1]:
# ! unzip /home/aistudio/data/data116120/常规赛：PALM眼底彩照中黄斑中央凹定位.zip -d /home/aistudio/work

In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from paddle.io import Dataset, DataLoader
import paddle.vision.transforms as T
import paddle.nn as nn
import paddle.nn.functional as F
import paddle
import random
from sklearn.model_selection import train_test_split
import math

# 读取数据分析分布

In [3]:
df = pd.read_excel('./work/dataset/Train/Fovea_Location_train.xlsx')
df['path'] = './work/dataset/Train/fundus_image/' + df['imgName']
df['size'] = df['path'].apply(lambda path: Image.open(path).size)
df['mode'] = df['path'].apply(lambda path: Image.open(path).mode)
df['nosee'] = df.apply(lambda row: 1 if (row['Fovea_X']+row['Fovea_Y'])<10 else 0, axis=1)
df.head(2)

,imgName,Fovea_X,Fovea_Y,path,size,mode,nosee
0,H0001.jpg,743.96,790.54,./work/dataset/Train/fundus_image/H0001.jpg,"(1444, 1444)",RGB,0
1,H0002.jpg,1394.82,725.54,./work/dataset/Train/fundus_image/H0002.jpg,"(1444, 1444)",RGB,0


In [4]:
print('图像大小\n', df['size'].value_counts())
print('图像模式\n', df['mode'].value_counts())

图像大小
 (2124, 2056)    694
(1444, 1444)    106
Name: size, dtype: int64
图像模式
 RGB    800
Name: mode, dtype: int64


In [5]:
df.sort_values('Fovea_X').head(50) # 不可见的样本

,imgName,Fovea_X,Fovea_Y,path,size,mode,nosee
295,P0109.jpg,0.00,0.00,./work/dataset/Train/fundus_image/P0109.jpg,"(1444, 1444)",RGB,1
684,V0285.jpg,0.00,0.00,./work/dataset/Train/fundus_image/V0285.jpg,"(1444, 1444)",RGB,1
342,P0156.jpg,0.00,0.00,./work/dataset/Train/fundus_image/P0156.jpg,"(2124, 2056)",RGB,1
639,V0240.jpg,0.00,0.00,./work/dataset/Train/fundus_image/V0240.jpg,"(1444, 1444)",RGB,1
647,V0248.jpg,0.00,0.00,./work/dataset/Train/fundus_image/V0248.jpg,"(2124, 2056)",RGB,1
11,H0012.jpg,0.00,0.00,./work/dataset/Train/fundus_image/H0012.jpg,"(1444, 1444)",RGB,1
347,P0161.jpg,288.67,556.93,./work/dataset/Train/fundus_image/P0161.jpg,"(1444, 1444)",RGB,0
346,P0160.jpg,467.50,740.84,./work/dataset/Train/fundus_image/P0160.jpg,"(1444, 1444)",RGB,0
327,P0141.jpg,613.11,651.45,./work/dataset/Train/fundus_image/P0141.jpg,"(1444, 1444)",RGB,0
489,V0090.jpg,654.53,767.43,./work/dataset/Train/fundus_image/V0090.jpg,"(1444, 1444)",RGB,0


In [6]:
df_test = pd.DataFrame(os.listdir('./work/dataset/PALM-Testing400-Images'), columns=['FileName'])
df_test = df_test.sort_values('FileName').reset_index(drop=True)
df_test['path'] = './work/dataset/PALM-Testing400-Images/' + df_test['FileName']
df_test['size'] = df_test['path'].apply(lambda path: Image.open(path).size)
df_test['mode'] = df_test['path'].apply(lambda path: Image.open(path).mode)
df_test.head(2)

,FileName,path,size,mode
0,T0001.jpg,./work/dataset/PALM-Testing400-Images/T0001.jpg,"(2124, 2056)",RGB
1,T0002.jpg,./work/dataset/PALM-Testing400-Images/T0002.jpg,"(2124, 2056)",RGB


# 基线程序

In [7]:
def get_angle(x, y):
    # 0 -> 360°
    ang = math.atan2(y, x)
    ang = ang * 180 / math.pi
    if ang > 360:
        ang -= 360
    if ang < 0:
        ang += 360
    return ang
get_angle(0,0.1)

90.0

In [8]:

class PALMDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.mode = mode
        self.df = df.reset_index(drop=True)
        self.nlen = len(df)
        # self.common_before_trans = T.Compose([
        #     T.Resize(size=(404,404))
        # ])
        self.common_after_trans = T.Compose([
            T.Resize(size=(404,404)),
            T.ToTensor(),
            T.Normalize(mean=(0.496,0.456,0.406), std=(0.229,0.224,0.225))
        ])
        self.enhance_op = [
            self.rotate_trans,
            self.colorjitter_trans,
            self.horizontalFlip_trans,
            self.verticalFlip_trans,
        ]
    
    def rotate_trans(self, img, xy):
        pos_x, pos_y = xy
        pos_x, pos_y = pos_x-0.5, pos_y-0.5 # 以中心为坐标原点
        if abs(pos_x)<1e-4 and abs(pos_y)<1e-4:
            return img, xy # 靠近旋转中心, 意义不大
        ang_0 = get_angle(pos_x, pos_y)
        lens = np.linalg.norm([pos_x, pos_y])
        try_times = 0
        while True:
            rot_ang = random.random()*60-30 # 随机旋转±30°
            new_ang = (ang_0 + rot_ang)/180.0 * math.pi
            new_pos_x = lens*math.cos(new_ang)
            new_pos_y = lens*math.sin(new_ang)
            if abs(new_pos_x)<0.45 or abs(new_pos_y)<0.45: # 取0.45, 避免落在边界
                pos_x = new_pos_x + 0.5
                pos_y = new_pos_y + 0.5
                break
            try_times += 1
            if try_times > 10: # 避免死循环
                pos_x += 0.5
                pos_y += 0.5
                rot_ang = 0
                break
        img = T.rotate(img, rot_ang)
        return img, (pos_x, pos_y)
    
    def colorjitter_trans(self, img, xy):
        img = T.ColorJitter(0.2,0.2,0.2,0.02)(img)
        return img, xy

    def horizontalFlip_trans(self, img, xy):
        img = T.hflip(img)
        pos_x, pos_y = xy
        return img, (1-pos_x, pos_y)
    
    def verticalFlip_trans(self, img, xy):
        img = T.vflip(img)
        pos_x, pos_y = xy
        return img, (pos_x, 1-pos_y)

    
    def enhance(self, img, pos: tuple):
        # img = self.common_before_trans(img)
        if self.mode == 'train':
            for trans_fun in self.enhance_op:
                # print(trans_fun, pos_fun)
                if random.random()>0.5:
                    img, pos = trans_fun(img, pos)
        img = self.common_after_trans(img)
        return img, pos


    def __len__(self):
        return self.nlen

    def __getitem__(self, idx):
        path = self.df.loc[idx, 'path']
        img = Image.open(path)
        pos = (0, 0)
        if self.mode == 'train':
            width, height = self.df.loc[idx, 'size']
            raw_posx, raw_posy = self.df.loc[idx, 'Fovea_X'], self.df.loc[idx, 'Fovea_Y']
            pos = (raw_posx/width, raw_posy/height)
        img, pos = self.enhance(img, pos)
        return img.astype('float32'), np.array(pos, dtype='float32')

df_train, df_dev = train_test_split(df, test_size=0.2, stratify=df['nosee'], random_state=76)
print(df_train.shape, df_dev.shape)

train_ds = PALMDataset(df_train, 'train')
dev_ds = PALMDataset(df_dev, 'dev')
print(len(train_ds), len(dev_ds))
print(train_ds[10][0].shape, train_ds[10][1])

(640, 7) (160, 7)
640 160


W1111 11:43:33.198629  4869 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1111 11:43:33.203009  4869 device_context.cc:465] device: 0, cuDNN Version: 7.6.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


[3, 404, 404] [0.48403355 0.46523488]


In [9]:

class PAMLNet(nn.Layer):
    def __init__(self):
        super(PAMLNet, self).__init__()
        self.backbone = paddle.vision.mobilenet_v2(pretrained=False, num_classes=-1)
        self.fc = nn.Sequential(
            nn.Flatten(1),
            nn.Dropout(0.1),
            nn.Linear(1280, 64),
            nn.ELU(),
            nn.Linear(64, 2),
        )
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)
        return x

model = paddle.Model(PAMLNet())
# model.summary((1, 3,404,608))

In [10]:
# paddle.summary(paddle.vision.resnet101(pretrained=False, num_classes=-1), (1, 3, 204,204))

In [11]:
class PALMMetric(paddle.metric.Metric):
    def __init__(self, name='palm', *args, **kwargs):
        super(PALMMetric, self).__init__(*args, **kwargs)
        self._name = name
        self.reset()


    def update(self, pred, label):
        if self.pred_pos is None:
            self.pred_pos = pred
            self.real_pos = label
        else:
            self.pred_pos = np.r_[self.pred_pos, pred]
            self.real_pos = np.r_[self.real_pos, label]

    def reset(self):
        self.pred_pos = None
        self.real_pos = None

    def accumulate(self):
        err_dis = np.linalg.norm(self.pred_pos-self.real_pos, axis=1).mean()
        return 1.0/err_dis

    def name(self):
        return self._name

In [12]:
schedual = paddle.optimizer.lr.CosineAnnealingDecay(5e-4, T_max=80)

In [13]:
model.prepare(
    optimizer=paddle.optimizer.Adam(parameters=model.parameters(), learning_rate=schedual),
    loss=nn.SmoothL1Loss(),
    metrics=PALMMetric()
)

In [14]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
dev_loader = DataLoader(dev_ds, batch_size=16, shuffle=False, num_workers=2)

model.fit(
    train_loader,
    dev_loader,
    epochs=50,
    batch_size=16,
    save_dir='./work/model',
    save_freq=10,
    callbacks=[paddle.callbacks.EarlyStopping(monitor='palm',save_best_model=True, patience=20)]
)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step 10/40 - loss: 0.0410 - palm: 2.3457 - 3s/step
step 20/40 - loss: 0.0588 - palm: 2.5494 - 3s/step
step 30/40 - loss: 0.0255 - palm: 2.6585 - 3s/step
step 40/40 - loss: 0.0441 - palm: 2.7866 - 3s/step
save checkpoint at /home/aistudio/work/model/0
Eval begin...
step 10/10 - loss: 0.0013 - palm: 13.9486 - 756ms/step
Eval samples: 160
Epoch 2/50
step 10/40 - loss: 0.0400 - palm: 3.1637 - 3s/step
step 20/40 - loss: 0.0277 - palm: 3.0983 - 3s/step
step 30/40 - loss: 0.0442 - palm: 3.1389 - 3s/step
step 40/40 - loss: 0.0334 - palm: 3.1623 - 3s/step
Eval begin...
step 10/10 - loss: 0.0457 - palm: 2.3529 - 775ms/step
Eval samples: 160
Epoch 3/50
step 10/40 - loss: 0.0380 - palm: 3.3155 - 3s/step
step 20/40 - loss: 0.0355 - palm: 3.2055 - 3s/step
step 30/40 - loss: 0.0285 - palm: 3.2359 - 3s/step
step 40/40 - loss: 0.0346 - palm: 3.2794 - 3s/step
Eval begin...
step 10/10 - loss: 0.1825 - palm: 1.2296 - 757ms/step
Eval samples: 160
Epoch 4/50
step 10/40 - loss: 0.0306 - palm: 3.3544 - 3s/ste

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 583, in _get_data
    data = self._data_queue.get(timeout=self._timeout)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/queues.py", line 105, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 505, in _thread_loop
    batch = self._get_data()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/

KeyboardInterrupt: 

In [ ]:
model.save('./work/11111036')

In [15]:
test_ds = PALMDataset(df_test, 'test')
print(len(test_ds))
print(test_ds[0][1])

400
[0. 0.]


In [16]:
y_pred = model.predict(test_ds, batch_size=32, stack_outputs=True)[0]

Predict begin...
step 13/13 [==============================] - 3s/step          
Predict samples: 400


In [17]:
df_test[['Fovea_X','Fovea_Y']] =  y_pred

In [18]:
df_test['Fovea_X'] *= df_test['size'].apply(lambda x: x[0])
df_test['Fovea_Y'] *= df_test['size'].apply(lambda x: x[1])

In [ ]:
df_test.head(2)

In [19]:
df_test[['FileName','Fovea_X','Fovea_Y']].to_csv('Fovea_Localization_Results_3.csv', index=False)